In [4]:
!python -m pip install -q autogluon;

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 9.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.1/225.1 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.9/454.9 kB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.3/487.3 kB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.7/189.7 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.0/71.0 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 12.2 MB/s eta 0:0

In [ ]:
import pandas as pd
import numpy as np
import time
import shutil
import warnings
from autogluon.tabular import TabularPredictor

warnings.filterwarnings('ignore')

TIME_LIMIT_PER_MODEL = 1800 
IMPORTANCE_THRESHOLD_RATIO = 0.01 
TARGET_LABELS = ['Y1', 'Y2']

print("Loading data...")
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test (1).csv')

test_ids = test_df['id']

test_df = test_df.drop('id', axis=1)

train_df = train_df.sort_values(by='time').reset_index(drop=True)
test_df = test_df.sort_values(by='time').reset_index(drop=True)

def build_features(df):
    print("Building time-series features...")
    df_featured = df.copy()
    core_features = [col for col in df.columns if 'A' <= col <= 'N']

    for feature in core_features:
        for i in [1, 2, 3, 5, 10]:
            df_featured[f'{feature}_lag{i}'] = df_featured[feature].shift(i)
        df_featured[f'{feature}_diff1'] = df_featured[feature].diff(1)
        # Rolling Windows
        for window in [3, 5, 10]:
            df_featured[f'{feature}_rolling_mean_{window}'] = df_featured[feature].shift(1).rolling(window=window, min_periods=1).mean()
            df_featured[f'{feature}_rolling_std_{window}'] = df_featured[feature].shift(1).rolling(window=window, min_periods=1).std()

    df_featured = df_featured.fillna(method='ffill').fillna(method='bfill')

    print(f"New feature count: {len(df_featured.columns)}")
    return df_featured

combined_df = pd.concat([train_df.drop(columns=TARGET_LABELS), test_df], ignore_index=True)
combined_df_featured = build_features(combined_df)

train_df_featured = combined_df_featured.iloc[:len(train_df)].copy()
test_df_featured = combined_df_featured.iloc[len(train_df):].copy()

train_df_featured[TARGET_LABELS] = train_df[TARGET_LABELS]


def get_top_features(df, target, importance_threshold_ratio=0.01):
    temp_model_path = f"temp_feature_model_{target}_{int(time.time())}"
    print(f"  Training temporary model for {target} to find important features...")
    other_targets = [col for col in TARGET_LABELS if col != target]
    feature_selection_data = df.drop(columns=other_targets)

    try:
        feature_predictor = TabularPredictor(label=target, path=temp_model_path, eval_metric='mean_absolute_error')\
            .fit(train_data=feature_selection_data, hyperparameters={'GBM': {}}, time_limit=60)
        importances = feature_predictor.feature_importance(data=feature_selection_data)
        if not importances.empty:
            max_importance = importances['importance'].max()
            threshold = importance_threshold_ratio * max_importance
            selected_features = importances[importances['importance'] >= threshold].index.tolist()
        else: selected_features = []
    finally:
        if 'feature_predictor' in locals() and feature_predictor.path:
            shutil.rmtree(feature_predictor.path)
            print(f"  Cleaned up temporary directory: {temp_model_path}")

    return selected_features

all_predictions = {}
validation_scores = {} 

for target_label in TARGET_LABELS:
    print(f"\n--- Processing Target: {target_label} ---")

    important_features = get_top_features(train_df_featured, target_label, IMPORTANCE_THRESHOLD_RATIO)

    if not important_features:
        print(f"  Warning: No features met the importance threshold for {target_label}. Using original features.")
        important_features = [col for col in train_df.columns if 'A' <= col <= 'N' or col == 'time']

    print(f"Selected {len(important_features)} Features for {target_label}: {important_features}")

    train_subset = train_df_featured[important_features + [target_label]]
    test_subset = test_df_featured[important_features]

    main_model_path = f"AutogluonModels_{target_label}"
    print(f"  Training main model for {target_label}. Results will be in '{main_model_path}'")

    predictor = TabularPredictor(
        label=target_label,
        path=main_model_path,
        eval_metric='r2' 
    ).fit(
        train_data=train_subset,
        time_limit=TIME_LIMIT_PER_MODEL,
        presets='best_quality',
        hyperparameters={'GBM': {}, 'CAT': {}, 'XGB': {}, 'RF': {}, 'XT': {}}
    )

    print(f"  Predicting {target_label} on test data...")
    all_predictions[target_label] = predictor.predict(test_subset)

    print(f"  Getting validation R² score for {target_label}...")
    leaderboard = predictor.leaderboard(train_subset, silent=True)
    best_model_score = leaderboard.iloc[0]['score_val']
    validation_scores[target_label] = best_model_score

    print(f"  Leaderboard for {target_label}:")
    print(leaderboard)


print("\n--- Calculating Final Validation R² Scores ---")

if 'Y1' in validation_scores and 'Y2' in validation_scores:
    r2_y1 = validation_scores['Y1']
    r2_y2 = validation_scores['Y2']

    final_r2_score = (r2_y1 + r2_y2) / 2

    print(f"Validation R² Score for Y1 (from Leaderboard): {r2_y1:.4f}")
    print(f"Validation R² Score for Y2 (from Leaderboard): {r2_y2:.4f}")
    print(f"Final Average Validation R² Score: {final_r2_score:.4f}")
else:
    print("Could not calculate final scores because one or both targets were not processed.")


print("\n--- Creating Final Submission File ---")

final_predictions_df = pd.DataFrame(all_predictions)
final_predictions_df['id'] = test_ids
final_predictions_df = final_predictions_df[['id'] + TARGET_LABELS]

final_predictions_df.to_csv('submission_autogluon.csv', index=False)

print("Submission file created: submission_autogluon_FINAL.csv")
print(final_predictions_df.head())

Loading data...
Building time-series features...


Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.4.0
Python Version:     3.12.11
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Sun Mar 30 16:01:29 UTC 2025
CPU Count:          2
Memory Avail:       6.87 GB / 12.67 GB (54.2%)
Disk Space Avail:   62.32 GB / 107.72 GB (57.9%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets. Defaulting to `'medium'`...
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='extreme' : New in v1.4: Massively better than 'best' on datasets <30000 samples by using new models meta-learned on https://tabarena.ai: TabPFNv2, TabICL, Mitra, and TabM. Absolute best accuracy. Requires a GPU. Recommended 64 GB CPU memory and 32+ GB GPU memory.
	presets='best'    : Maximize accuracy. Recommended for most users. Use in c

New feature count: 183

--- Processing Target: Y1 ---
  Training temporary model for Y1 to find important features...


Beginning AutoGluon training ... Time limit = 60s
AutoGluon will save models to "/content/temp_feature_model_Y1_1758546400"
Train Data Rows:    80000
Train Data Columns: 183
Label Column:       Y1
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (29.859458454056327, -28.9181795461607, -0.00281, 0.97066)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during Predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression', 'quantile'])
Problem Type:       regression
Preprocessing data ...
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    7124.23 MB
	Train Data (Original)  Memory Usage: 111.69 MB (1.6% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadat

  Cleaned up temporary directory: temp_feature_model_Y1_1758546400
Selected 14 Features for Y1: ['G', 'H', 'J', 'M', 'E', 'C', 'time', 'N', 'F_diff1', 'G_diff1', 'A_diff1', 'K_diff1', 'M_lag1', 'K']
  Training main model for Y1. Results will be in 'AutogluonModels_Y1'


...

  Leaderboard for Y1:
                 model  score_test  score_val eval_metric  pred_time_test  \
0  RandomForest_BAG_L1    0.961027   0.746777          r2        1.435604   
1  WeightedEnsemble_L3    0.866156   0.765925          r2       11.269676   
2  WeightedEnsemble_L2    0.865871   0.765901          r2        7.562623   
3      LightGBM_BAG_L2    0.832428   0.760683          r2       11.264822   
4      LightGBM_BAG_L1    0.811237   0.763108          r2        6.121696   

   pred_time_val    fit_time  pred_time_test_marginal  pred_time_val_marginal  \
0       0.872779  105.916589                 1.435604                0.872779   
1       2.690001  202.190048                 0.004854                0.003322   
2       2.082565  157.613549                 0.005322                0.003343   
3       2.686679  201.993702                 3.707521                0.607456   
4       1.206444   51.577542                 6.121696                1.206444   

   fit_time_marginal  stack_

Beginning AutoGluon training ... Time limit = 60s
AutoGluon will save models to "/content/temp_feature_model_Y2_1758547051"
Train Data Rows:    80000
Train Data Columns: 183
Label Column:       Y2
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (34.63603924182771, -0.8507991164008696, -0.06117, 0.9237)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during Predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression', 'quantile'])
Problem Type:       regression
Preprocessing data ...
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    6968.89 MB
	Train Data (Original)  Memory Usage: 111.69 MB (1.6% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadat

  Cleaned up temporary directory: temp_feature_model_Y2_1758547051
Selected 40 Features for Y2: ['A', 'time', 'K', 'D', 'B', 'F', 'L', 'I', 'G', 'A_diff1', 'N_rolling_std_10', 'F_diff1', 'I_rolling_mean_10', 'A_lag1', 'A_rolling_mean_3', 'J_rolling_std_10', 'F_lag10', 'D_rolling_mean_10', 'E_rolling_mean_10', 'M_rolling_std_10', 'K_rolling_mean_10', 'G_rolling_mean_3', 'L_rolling_std_10', 'B_rolling_std_10', 'C', 'G_rolling_mean_5', 'B_lag10', 'H', 'L_rolling_mean_10', 'H_rolling_mean_10', 'K_diff1', 'D_diff1', 'M_rolling_mean_10', 'G_rolling_std_5', 'B_rolling_mean_10', 'L_rolling_mean_3', 'I_diff1', 'A_lag5', 'G_rolling_std_10', 'J_rolling_mean_10']
  Training main model for Y2. Results will be in 'AutogluonModels_Y2'


Leaderboard on holdout data (DyStack):
                 model  score_holdout  score_val eval_metric  pred_time_test  pred_time_val   fit_time  pred_time_test_marginal  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0      LightGBM_BAG_L1       0.797997   0.793257          r2        4.240934       4.115792  82.320615                 4.240934                4.115792          82.320615            1       True          1
1  WeightedEnsemble_L3       0.797997   0.793257          r2        4.243035       4.122634  82.336178                 0.002101                0.006842           0.015563            3       True          3
2  WeightedEnsemble_L2       0.797997   0.793257          r2        4.243443       4.119048  82.327836                 0.002509                0.003256           0.007221            2       True          2
	1	 = Optimal   num_stack_levels (Stacked Overfitting Occurred: False)
	99s	 = DyStack   runtime |	261s	 = Remaining runtime
Starting mai

  Predicting Y2 on test data...
  Getting validation R² score for Y2...
  Leaderboard for Y2:
                 model  score_test  score_val eval_metric  pred_time_test  \
0      LightGBM_BAG_L1    0.928964   0.805913          r2       68.857419   
1  WeightedEnsemble_L2    0.928964   0.805913          r2       68.860964   
2  WeightedEnsemble_L3    0.928964   0.805913          r2       68.861440   
3      LightGBM_BAG_L2    0.914892   0.793598          r2       73.174574   

   pred_time_val    fit_time  pred_time_test_marginal  pred_time_val_marginal  \
0      15.852228  178.131537                68.857419               15.852228   
1      15.855789  178.139014                 0.003545                0.003561   
2      15.854463  178.215857                 0.004022                0.002235   
3      16.761705  244.784099                 4.317156                0.909477   

   fit_time_marginal  stack_level  can_infer  fit_order  
0         178.131537            1       True          1 

In [15]:
sub = pd.read_csv('submission_autogluon.csv')
print(sub.shape)

(15996, 3)


In [17]:
# Replace NaN values in 'id' column with a sequence of integers starting from 1
sub['id'] = range(1, len(sub) + 1)

# Display the updated DataFrame
print(sub['id'].head())

0    1
1    2
2    3
3    4
4    5
Name: id, dtype: int64
